In [1]:
%load_ext autoreload
%autoreload 2

import sys, os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from meta_neural_network_architectures import VGGReLUNormNetwork, ResNet12
from prompters import padding
from utils.parser_utils import get_args

import easydict

import torch
import torch.nn as nn
import numpy as np

import torch.backends.cudnn as cudnn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim

from loss import *

In [2]:
os.environ['DATASET_DIR'] = os.path.join(os.getcwd(), "datasets")

args = easydict.EasyDict(
{
  "batch_size":2,
  "image_height":84,
  "image_width":84,
  "image_channels":3,
  "gpu_to_use":0,
  "num_dataprovider_workers":4,
  "max_models_to_save":5,
  "dataset_name":"mini_imagenet_full_size",
  "dataset_path":"mini_imagenet_full_size",
  "reset_stored_paths":False,
  "experiment_name":"alfa+maml",
  "train_seed": 0, "val_seed": 0,
  "indexes_of_folders_indicating_class": [-3, -2],
  "sets_are_pre_split": True,
  "train_val_test_split": [0.64, 0.16, 0.20],
  "evaluate_on_test_set_only": False,

  "total_epochs": 100,
  "total_iter_per_epoch":500, "continue_from_epoch": -2,
  "num_evaluation_tasks":600,
  "multi_step_loss_num_epochs": 15,
  "minimum_per_task_contribution": 0.01,
  "learnable_per_layer_per_step_inner_loop_learning_rate": False,
  "enable_inner_loop_optimizable_bn_params": False,
  "evalute_on_test_set_only": False,

  "max_pooling": True,
  "per_step_bn_statistics": False,
  "learnable_batch_norm_momentum": False,
  "load_into_memory": False,
  "init_inner_loop_learning_rate": 0.01,
  "init_inner_loop_weight_decay": 0.0005,
  "learnable_bn_gamma": True,
  "learnable_bn_beta": True,

  "dropout_rate_value":0.0,
  "min_learning_rate":0.001,
  "meta_learning_rate":0.001,   "total_epochs_before_pause": 100,
  "first_order_to_second_order_epoch":-1,
  "weight_decay": 0.0,

  "norm_layer":"batch_norm",
  "cnn_num_filters":48,
  "num_stages":4,
  "conv_padding": True,
  "number_of_training_steps_per_iter":5,
  "number_of_evaluation_steps_per_iter":5,
  "cnn_blocks_per_stage":1,
  "num_classes_per_set":5,
  "num_samples_per_class":5,
  "num_target_samples": 15,
    "samples_per_iter" : 1,

  "second_order": True,
  "use_multi_step_loss_optimization":False,
  "attenuate": False,
  "alfa": True,
  "random_init": False,
  "backbone": "4-CONV",
   "loss_function": "Softmax",
  "ole": True,
  "arbiter": False
}
)

device = torch.cuda.current_device()
args.im_shape = (2, 3, args.image_height, args.image_width)

args.use_cuda = torch.cuda.is_available()
args.seed = 104
args.reverse_channels=False
args.labels_as_int=False
args.reset_stored_filepaths=False
args.num_of_gpus=1

def get_inner_loop_parameter_dict(params):

    param_dict = dict()
    for name, param in params:
        if param.requires_grad:
            param_dict[name] = param.to(device=device)

    return param_dict

In [3]:
preprocess = transforms.Compose([
    transforms.Resize(84),
    transforms.ToTensor()
])

train_dataset = CIFAR100("./data", transform=preprocess,
                          download=True, train=True)

val_dataset = CIFAR100("./data", transform=preprocess,
                        download=True, train=False)

train_loader = DataLoader(train_dataset,
                          batch_size=25, pin_memory=True,
                          num_workers=16, shuffle=True)

class_names = train_dataset.classes

images, targets = next(iter(train_loader))
images = images.to(device)

print(images.shape)
print(targets)

Files already downloaded and verified
Files already downloaded and verified


C:\Users\JM\anaconda3\envs\metal\lib\site-packages\torch\utils\data\dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
# # 무작위 이미지 데이터 생성 (배치 크기, 채널, 높이, 너비)
# batch_size = 25
# channels = 3  # RGB 이미지이므로 3개의 채널
# height, width = 84, 84  # 높이와 너비

# # 무작위 이미지 데이터 생성 (0과 1 사이의 무작위 값)
# images = torch.rand(batch_size, channels, height, width)

# # 무작위 레이블 데이터 생성 (예시를 위해 10개의 클래스)
# num_classes = 25
# targets = torch.randint(0, num_classes, (batch_size,))

# images.shape

In [ ]:
targets = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4])
targets = torch.Tensor(targets)
targets = targets.type(torch.LongTensor)
targets = targets.to(device)

print(images.shape)
print(targets)

In [ ]:
class MAMLFewShotClassifier(nn.Module):
    
    def __init__(self, im_shape, device, args):
        
        super(MAMLFewShotClassifier, self).__init__()
        self.args = args
        self.device = device
        self.batch_size = args.batch_size
        self.use_cuda = args.use_cuda
        self.im_shape = im_shape
        self.current_epoch = 0
        
        self.classifier = VGGReLUNormNetwork(im_shape=self.im_shape, num_output_classes=self.args.
                                                 num_classes_per_set,
                                                 args=args, device=device, meta_classifier=True).to(device=self.device)        
        
        self.optimizer = optim.Adam(self.trainable_parameters(), lr=args.meta_learning_rate, amsgrad=False)
        self.scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer=self.optimizer, T_max=self.args.total_epochs,
                                                              eta_min=self.args.min_learning_rate)
        
    def trainable_parameters(self):
        for param in self.parameters():
            if param.requires_grad:
                yield param
                
    def get_inner_loop_parameter_dict(self, params):
        
        param_dict = dict()
        for name, param in params:
            if param.requires_grad:
                param_dict[name] = param.to(device=device)

        return param_dict
                
        
    def forward(self, x, y):
        
        names_weights_copy = self.get_inner_loop_parameter_dict(self.classifier.named_parameters())
        
        names_weights_copy = {
                        name.replace('module.', ''): value.unsqueeze(0).repeat(
                            [1] + [1 for i in range(len(value.shape))]) for
                        name, value in names_weights_copy.items()}
        
        loss, preds = self.net_forward(x, y, names_weights_copy)
        
        return loss, preds
    
    
    def net_forward(self, x, y, names_weights_copy):
        
    
        preds = self.classifier.forward(x, params=names_weights_copy,num_step=4)
        
        loss = F.cross_entropy(input=preds, target=y)        
        
        return loss, preds
    
    

model = MAMLFewShotClassifier(args=args, device=device, im_shape=(2, 3, args.image_height, args.image_width))

In [ ]:
loss, preds = model.forward(images, targets)

In [ ]:
svd = 0

# 각 레이어의 가중치에 대한 SVD를 수행합니다
for name, param in model.named_parameters():
    if 'weight' in name:  # weight에 대해서만 SVD를 수행합니다
        if "norm_layer" not in name:
            if "linear" not in name:
                print(f'Layer: {name}')
                original_shape = param.shape
                u, s, v = torch.svd(param, some=False)  # SVD 수행
                svd = s
                print(f'original shape: {original_shape}')
                print(f'U matrix shape: {u.shape}')
                print(f'Singular values shape: {s.shape}')
                print(f'V transpose matrix shape: {v.shape}')
                print('------------------------------------')
                
#print(svd)